In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:

import pickle 
##Load data
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/Users/abhisheksinghal/Documents/garima/MLLearning/ubi/supp.csv')

df.head()

y = df.target
x = df
x = x.drop(['target', 'row_id'], axis=1)
print(x.shape)
print(y.shape)
filename = "/Users/abhisheksinghal/Documents/garima/MLLearning/ubi/mar_model.sav"
final = "/Users/abhisheksinghal/Documents/garima/MLLearning/ubi/final_model.sav"


import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.3,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
regressor = TabNetRegressor(**tabnet_params)

# load the model from disk
regressor = pickle.load(open(filename, 'rb'))



(942961, 302)
(942961,)
Device used : cpu


In [6]:
import time
tscv = TimeSeriesSplit(n_splits = 20)
rmse = []
CV_score_array    =[]
predictions_array =[]

for train_index, test_index in tscv.split(x):
    X_train, X_valid = x.iloc[train_index], x.iloc[test_index]
    y_train, y_valid = y[train_index], y[test_index]

    X_train      = X_train.to_numpy()
    y_train = y_train.to_numpy().reshape(-1, 1)
    
    X_valid      = X_valid.to_numpy()
    y_valid = y_valid.to_numpy().reshape(-1, 1)
    
    
    t= time.time()
    print("Running started", time.time())
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              max_epochs=100,
              eval_metric=['rmse'])
    print ("Round took minutes?" ,(time.time()-t)/60)
    predictions = regressor.predict(X_valid)
    pickle.dump(regressor, open(filename, 'wb'))
    CV_score_array.append(regressor.best_cost)
    print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )
    predictions_array.append(np.expm1(predictions))
    
pickle.dump(regressor, open(filename, 'wb'))

Running started 1653086533.5339499

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_0_rmse = 0.28887
Best weights from best epoch are automatically used!
Round took minutes? 1.7789286375045776
The CV score is 0.28887
Running started 1653086644.8223329

Early stopping occurred at epoch 22 with best_epoch = 12 and best_val_0_rmse = 0.28862
Best weights from best epoch are automatically used!
Round took minutes? 5.616154233614604
The CV score is 0.28875
Running started 1653086987.44821

Early stopping occurred at epoch 20 with best_epoch = 10 and best_val_0_rmse = 0.28911
Best weights from best epoch are automatically used!
Round took minutes? 10.258787349859874
The CV score is 0.28887
Running started 1653087613.6396108

Early stopping occurred at epoch 22 with best_epoch = 12 and best_val_0_rmse = 0.28895
Best weights from best epoch are automatically used!
Round took minutes? 12.525031201044719
The CV score is 0.28889
Running started 1653088373.049357

Early stoppin